# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションにおいて、ターゲットとなる単語を推測するためのシンプルなエージェントを実装することに焦点を当てています。具体的には、エージェントはゲーム内での異なる役割（質問者または回答者）を奪い合い、ターンごとに質問や推測を行います。

### 問題の内容
コンペティションは、言葉当てゲーム「20の質問」に基づいており、エージェントは限られた質問回数の中で答えを導き出すことが求められます。このNotebookでは、一連の質問を通じてターゲット単語を推測することを目的とした複数のエージェントが設計されています。

### 使用されている手法とライブラリ
- **Python 3環境**: Jupyter Notebookは、KaggleのPython 3環境で実行されており、多くの分析ライブラリが事前にインストールされています。
- **ライブラリ**: `numpy`（線形代数）、`pandas`（データ処理、CSVファイルの入出力）などがインポートされています。これらのライブラリはデータ処理や分析に役立ちます。
- **Kaggle Environments**: `kaggle_environments`ライブラリが使用されており、これにより「llm_20_questions」環境が作成され、エージェントの実行とデバッグが可能となります。

### 実装の概要
- **シンプルなエージェントの実装**: エージェントは、各ターンにおいて「ask」（質問）、および「guess」（推測）することを担当します。例えば、`simple_agent1`は「それはアヒルですか？」という質問をし、その後のターンで「アヒル」と推測します。
- **ゲームの実行**: 最後に、実装されたエージェントを用いてゲームを実行し、ゲームの状態を表示するために`env.render`が呼び出されています。

このNotebookは、シンプルな戦略を持つエージェントを使用して、20の質問形式のゲームにおける基本的な推測メカニズムを示し、環境でのエージェントの相互作用を検証するためのものです。

---


# 用語概説 
以下は、ノートブック内のコードセルに含まれる専門用語の簡単な解説です。初心者がつまずきそうなマイナーなものや特有のドメイン知識に焦点を当てています。

1. **エージェント**:
    - ゲーム内で行動を行うプログラムやAIのことを指します。このコンペでは質問者や回答者として振る舞い、特定の戦略に基づいて質問や推測を行います。

2. **ターンの種類 (`turnType`)**:
    - ゲームが進行する中で、エージェントが実行できる動作のタイプを表します。具体的には、質問する (`ask`)、推測する (`guess`)、答える (`answer`) の3種類があります。

3. **観察情報 (`obs`)**:
    - エージェントがゲームの状態を理解するために持っている情報です。この情報には、現在のターンの種類や過去の質問・回答などが含まれます。

4. **環境 (`environment`)**:
    - エージェントが活動する背景や設定を指します。このコンペでは、「llm_20_questions」という特定のルールに基づく環境でプレイします。

5. **デバッグモード (`debug=True`)**:
    - エラーや問題を特定するための特別なモードで、通常よりも詳細な情報を出力してくれます。プログラムの不具合を見つけたり、動作の確認を行ったりするときに役立ちます。

6. **`env.run`**:
    - 環境内でエージェント同士がゲームを実行するための関数です。設定されたエージェントを使って対戦を開始します。

7. **`env.render`**:
    - ゲームの状態や進捗を視覚的に表示するための関数です。実行中のゲームのビジュアル表現を提供します。

8. **Kaggle Environments**:
    - Kaggleが提供する、AIや機械学習のアルゴリズムをテストするための環境やフレームワークのことを指します。このライブラリを使うことで、様々なゲームやシミュレーションを簡単に実行することができます。

これらの用語を理解することで、ノートブック内のコードの意味や動作をより深く理解する手助けになるでしょう。

---


In [ ]:
# このPython 3環境は、多くの便利な分析ライブラリがインストールされています
# これはkaggle/python Dockerイメージによって定義されています: https://github.com/kaggle/docker-python
# 例えば、いくつかの便利なパッケージをロードします

import numpy as np # 線形代数
import pandas as pd # データ処理、CSVファイルの入出力 (例: pd.read_csv)

# 入力データファイルは、読み取り専用の"../input/"ディレクトリにあります
# 例えば、これを実行すると (実行ボタンをクリックするかShift + Enterを押すことで)、入力ディレクトリ内のすべてのファイルがリストされます

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# 現在のディレクトリ(/kaggle/working/)に最大20GBまで書き込むことができ、
# "すべてを保存して実行"を使用してバージョンを作成する際に出力として保存されます
# 一時ファイルは/kaggle/temp/に書き込むこともできますが、
# 現在のセッションの外では保存されません。

In [ ]:
def simple_agent1(obs, cfg):    
    # エージェントが質問者であり、ターンの種類が"ask"の場合
    if obs.turnType == "ask": response = "それはアヒルですか？"
    elif obs.turnType == "guess": response = "アヒル"
    elif obs.turnType == "answer": response = "いいえ"
    print(obs) # 観察情報を表示
    return response

def simple_agent2(obs, cfg):
    # エージェントが質問者であり、ターンの種類が"ask"の場合
    if obs.turnType == "ask": response = "それは鳥ですか？"
    elif obs.turnType == "guess": response = "鳥"
    elif obs.turnType == "answer": response = "いいえ"
    print(obs) # 観察情報を表示
    return response

def simple_agent3(obs, cfg):
    # エージェントが質問者であり、ターンの種類が"ask"の場合
    if obs.turnType == "ask": response = "それは豚ですか？"
    elif obs.turnType == "guess": response = "豚"
    elif obs.turnType == "answer": response = "いいえ"
    print(obs) # 観察情報を表示
    return response

def simple_agent4(obs, cfg):
    # エージェントが質問者であり、ターンの種類が"ask"の場合
    if obs.turnType == "ask": response = "それは牛ですか？"
    elif obs.turnType == "guess": response = "牛"
    elif obs.turnType == "answer": response = "いいえ"
    print(obs) # 観察情報を表示
    return response

In [ ]:
from kaggle_environments import make
env = make(environment="llm_20_questions", debug=True) # "llm_20_questions"環境を作成し、デバッグモードを有効にします

In [ ]:
game_output = env.run(agents=[simple_agent1, simple_agent3, simple_agent2, simple_agent4]) # エージェントを使ってゲームを実行します

In [ ]:
env.render(mode="ipython", width=400, height=400) # ゲームの状態を表示します